In [4]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Load

In [5]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
df_hist = utils.read_df_pkl('../input/histori*0*')
df_new = utils.read_df_pkl('../input/new_mer*0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
df_hist  = reduce_mem_usage(df_hist )
df_new   = reduce_mem_usage(df_new  )

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1971.22 Mb (0.0% reduction)
Mem. usage decreased to 129.17 Mb (0.0% reduction)


In [6]:
df_hist_trans = df_hist
df_new_merchant_trans = df_new

In [4]:
# for df in [df_hist_trans,df_new_merchant_trans]:
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)

In [7]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [8]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
#     df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
#     df['month'] = df['purchase_date'].dt.month
#     df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
#     df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
#     df['month_diff'] += df['month_lag']

In [11]:
aggs = {}
# for col in ['month','hour','weekofyear','dayofweek','subsector_id','merchant_id','merchant_category_id']:
for col in ['weekofyear',  'merchant_id']:
    aggs[col] = ['nunique']

# aggs['purchase_amount'] = ['sum','max','min','mean','var']
# aggs['purchase_amount'] = ['max','mean']
# aggs['installments'] = ['max','min','mean']
aggs['purchase_date'] = ['max','min']
# aggs['month_lag'] = ['max','mean']
# aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# for col in ['category_2','category_3']:
#     df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
#     aggs[col+'_mean'] = ['mean']    

new_columns = get_new_columns('hist',aggs)
df_hist_trans_group = df_hist_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group;gc.collect()

127

In [12]:
aggs = {}
# for col in ['month','hour','weekofyear','dayofweek','subsector_id','merchant_id','merchant_category_id']:
for col in ['weekofyear',  'merchant_id']:
    aggs[col] = ['nunique']

# aggs['purchase_amount'] = ['sum','max','min','mean','var']
# aggs['purchase_amount'] = ['max','mean']
# aggs['installments'] = ['max','min','mean']
aggs['purchase_date'] = ['max','min']
# aggs['month_lag'] = ['max','mean']
# aggs['month_diff'] = ['mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['weekend'] = ['sum', 'mean']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

# for col in ['category_2','category_3']:
#     df_hist_trans[col+'_mean'] = df_hist_trans.groupby([col])['purchase_amount'].transform('mean')
#     aggs[col+'_mean'] = ['mean']    
    
new_columns = get_new_columns('new',aggs)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_purchase_date_diff'] = (df_hist_trans_group['new_purchase_date_max'] - df_hist_trans_group['new_purchase_date_min']).dt.days
df_hist_trans_group['new_purchase_date_average'] = df_hist_trans_group['new_purchase_date_diff']/df_hist_trans_group['new_card_id_size']
df_hist_trans_group['new_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_purchase_date_max']).dt.days
df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()

147

In [13]:
del df_hist_trans;gc.collect()
del df_new_merchant_trans;gc.collect()
df_train.head(5)

,card_id,first_active_month,feature_1,feature_2,feature_3,target,hist_weekofyear_nunique,hist_merchant_id_nunique,hist_purchase_date_max,hist_purchase_date_min,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,new_weekofyear_nunique,new_merchant_id_nunique,new_purchase_date_max,new_purchase_date_min,new_authorized_flag_sum,new_authorized_flag_mean,new_weekend_sum,new_weekend_mean,new_category_1_sum,new_category_1_mean,new_card_id_size,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow
0,C_ID_92a2005557,2017-06,5,2,1,-0.820312,35,94,2018-02-25 09:31:15,2017-06-27 14:18:08,247,0.950000,90,0.346154,0,0.000000,260,242,0.930769,348,7.0,23.0,2018-04-29 11:23:05,2018-03-05 14:04:36,23.0,1.0,6.0,0.260870,0.0,0.000000,23.0,54.0,2.347826,285.0
1,C_ID_3d0044924f,2017-01,4,1,0,0.392822,50,142,2018-01-31 22:31:09,2017-01-06 16:29:42,339,0.968571,132,0.377143,31,0.088571,350,390,1.114286,373,4.0,6.0,2018-03-30 06:48:26,2018-02-01 17:07:54,6.0,1.0,0.0,0.000000,0.0,0.000000,6.0,56.0,9.333333,316.0
2,C_ID_d639edf6cd,2016-08,2,2,0,0.687988,22,13,2018-02-27 19:08:25,2017-01-11 08:21:22,41,0.953488,11,0.255814,0,0.000000,43,412,9.581395,346,1.0,1.0,2018-04-28 17:43:11,2018-04-28 17:43:11,1.0,1.0,1.0,1.000000,0.0,0.000000,1.0,0.0,0.000000,286.0
3,C_ID_186d6a6901,2017-09,4,3,0,0.142456,20,50,2018-02-28 11:44:40,2017-09-26 16:22:21,77,1.000000,11,0.142857,12,0.155844,77,154,2.000000,345,5.0,7.0,2018-04-18 11:00:11,2018-03-07 11:55:06,7.0,1.0,3.0,0.428571,1.0,0.142857,7.0,41.0,5.857143,296.0
4,C_ID_cdbd2c0db2,2017-11,1,3,0,-0.159790,17,66,2018-02-28 20:40:41,2017-11-12 00:00:00,128,0.962406,42,0.315789,15,0.112782,133,108,0.812030,345,8.0,36.0,2018-04-28 18:50:25,2018-03-02 11:55:43,36.0,1.0,12.0,0.333333,2.0,0.055556,36.0,57.0,1.583333,286.0


In [10]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [14]:
for df in [df_train, df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
#     df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
#     df['new_first_buy'] = (df['new_purchase_date_min'] - df['first_active_month']).dt.days
#     for f in ['hist_purchase_date_max','hist_purchase_date_min','new_purchase_date_max',\
#                      'new_purchase_date_min']:
#         df[f] = df[f].astype(np.int64) * 1e-9
#     df['card_id_total'] = df['new_card_id_size'].fillna(0)+df['hist_card_id_size'].fillna(0)
#     df['purchase_amount_total'] = df['new_purchase_amount_sum']+df['hist_purchase_amount_sum']

# for f in ['feature_1','feature_2','feature_3']:
#     order_label = df_train.groupby([f])['outliers'].mean()
#     df_train[f] = df_train[f].map(order_label)
#     df_test[f] = df_test[f].map(order_label)

In [21]:
df_train_columns = [c for c in df_train.columns if c not in ['card_id', 'first_active_month','target','outliers']]
target = df_train['target']
del df_train['target']

In [23]:
df_train.head()

,card_id,first_active_month,feature_1,feature_2,feature_3,hist_weekofyear_nunique,hist_merchant_id_nunique,hist_purchase_date_max,hist_purchase_date_min,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_weekend_sum,hist_weekend_mean,hist_category_1_sum,hist_category_1_mean,hist_card_id_size,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,new_weekofyear_nunique,new_merchant_id_nunique,new_purchase_date_max,new_purchase_date_min,new_authorized_flag_sum,new_authorized_flag_mean,new_weekend_sum,new_weekend_mean,new_category_1_sum,new_category_1_mean,new_card_id_size,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,dayofweek,weekofyear,month,hist_first_buy
0,C_ID_92a2005557,2017-06-01,5,2,1,35,94,2018-02-25 09:31:15,2017-06-27 14:18:08,247,0.950000,90,0.346154,0,0.000000,260,242,0.930769,348,7.0,23.0,2018-04-29 11:23:05,2018-03-05 14:04:36,23.0,1.0,6.0,0.260870,0.0,0.000000,23.0,54.0,2.347826,285.0,3,22,6,26
1,C_ID_3d0044924f,2017-01-01,4,1,0,50,142,2018-01-31 22:31:09,2017-01-06 16:29:42,339,0.968571,132,0.377143,31,0.088571,350,390,1.114286,373,4.0,6.0,2018-03-30 06:48:26,2018-02-01 17:07:54,6.0,1.0,0.0,0.000000,0.0,0.000000,6.0,56.0,9.333333,316.0,6,52,1,5
2,C_ID_d639edf6cd,2016-08-01,2,2,0,22,13,2018-02-27 19:08:25,2017-01-11 08:21:22,41,0.953488,11,0.255814,0,0.000000,43,412,9.581395,346,1.0,1.0,2018-04-28 17:43:11,2018-04-28 17:43:11,1.0,1.0,1.0,1.000000,0.0,0.000000,1.0,0.0,0.000000,286.0,0,31,8,163
3,C_ID_186d6a6901,2017-09-01,4,3,0,20,50,2018-02-28 11:44:40,2017-09-26 16:22:21,77,1.000000,11,0.142857,12,0.155844,77,154,2.000000,345,5.0,7.0,2018-04-18 11:00:11,2018-03-07 11:55:06,7.0,1.0,3.0,0.428571,1.0,0.142857,7.0,41.0,5.857143,296.0,4,35,9,25
4,C_ID_cdbd2c0db2,2017-11-01,1,3,0,17,66,2018-02-28 20:40:41,2017-11-12 00:00:00,128,0.962406,42,0.315789,15,0.112782,133,108,0.812030,345,8.0,36.0,2018-04-28 18:50:25,2018-03-02 11:55:43,36.0,1.0,12.0,0.333333,2.0,0.055556,36.0,57.0,1.583333,286.0,2,44,11,11


In [26]:
# import sys
# sys.path.append('../../data_analysis/library/')
# import utils
prefix = '311_ker'
prefix = '411_ker'

ignore_features = ['first_active_month', 'card_id', 'outlier', 'target']

df_feat = pd.concat([df_train, df_test], axis=0)

for col in df_train_columns:
    if col in ignore_features:
        continue
    if col.count('purchase_date_min') or col.count('purchase_date_max'):
        continue
    if col.count('feature_'):continue
#     if not(col.count('card_id_total')):continue

    try:
        feature = df_feat[col].astype('float32').values
        feature = df_feat[col].fillna(-1).astype('float32').values
    except TypeError:
        print(col)
        sys.exit()
    col = col.replace('.', '_')

    utils.to_pkl_gzip(path = f'../features/1_first_valid/{prefix}_{col}@', obj=feature)